In [24]:
ls ../../../

Banjo/                       logo_homog_image_count.json
Learning/                    meetup/
Trading_Env/                 mirror_logos/
automated_trading/           speach/
automated_trading_client/    ta-lib/
candles.json                 test_candle_data.json
classifier_data.json         tests/
gym_data/


In [6]:
import datetime
import json
import time

from pytz import timezone 

In [7]:
def load_text_file(t_file):
    for candle in open(t_file):
        candle = candle.rstrip('\n')
        yield candle

In [8]:
def split_candle_text(candles):
    fields = ['date', 'time', 'open', 'high', 'close', 'low']
    for candle in candles:
        data = candle.split(',')
        candle_data = {}
        for ix, field in enumerate(fields):
            candle_data[field] = data[ix]
        yield candle_data
    

In [9]:
def format_time(date, time):
    date_parts = date.split('.')
    time_parts = time.split(':')
    
    year = date_parts[0]
    month = date_parts[1]
    day = date_parts[2]
    hour = time_parts[0]
    minute = time_parts[1]
    
    return year, month, day, hour, minute

In [10]:
def make_utc_time(year, month, day, hour, minute, timezone):
    target_time = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), tzinfo=timezone)
    epoch = datetime.datetime(1970,1,1, tzinfo=timezone)
    return (target_time - epoch).total_seconds()
    

In [21]:
def format_candles(candles, timezone):
    for candle in candles:
        year, month, day, hour, minute = format_time(candle['date'], candle['time'])
        utc = make_utc_time(year, month, day, hour, minute, timezone)
        candle = {'high': float(candle['high']), 'low': float(candle['low']), 'close': float(candle['close']), 'open': float(candle['open'])}
        date = {'year': int(year), 'month': int(month), 'day': int(day) , 'hour': int(hour), 'minute': int(minute), 'utc': utc}
        _id = time.time()
        candle_datum = {
            'candle': candle,
            'date': date,
            'id': _id
        }
        yield candle_datum

In [22]:
def dump_candle_data(data, destination):
    with open(destination, 'w') as f:
        json.dump(data, f)

In [23]:
def get_candle_data(input_file, output_file):
    EST_TIMEZONE = timezone('US/Eastern') # East timezone without daylight savings
    candle_data = load_text_file(input_file)
    split_candle_data = split_candle_text(candle_data)
    formatted_candle_data = list(format_candles(split_candle_data, EST_TIMEZONE))
    dump_candle_data(formatted_candle_data, output_file)
    

In [24]:
input_file = '/Users/jnewman/Desktop/DAT_MT_EURUSD_M1_2015.txt'
output_file = 'DAT_MT_EURUSD_M1_2015.json'

In [25]:
get_candle_data(input_file, output_file)